In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)
# openai = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="ollama")
openai = OpenAI()

In [ ]:
reader = PdfReader("resources/linkedin.pdf")
LINKEDIN = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        LINKEDIN += text

In [ ]:
with open("resources/summary.txt", "r", encoding="utf-8") as f:
    SUMMARY = f.read()

In [ ]:
NAME = "Prasun Bhattacharyya"

In [ ]:
SYSTEM_PROMPT = f"You are acting as {NAME}. You are answering questions on {NAME}'s website, \
particularly questions related to {NAME}'s career, background, skills and experience. \
Your responsibility is to represent {NAME} for interactions on the website as faithfully as possible. \
You are given a summary of {NAME}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

SYSTEM_PROMPT += f"\n\n## Summary:\n{SUMMARY}\n\n## LinkedIn Profile:\n{LINKEDIN}\n\n"
SYSTEM_PROMPT += f"With this context, please chat with the user, always staying in character as {NAME}."


In [ ]:
# Create a Pydantic model for the Evaluation
class Evaluation(BaseModel):
    """
    A Pydantic model for the evaluation of a response to a question.
    Args:
        is_acceptable (bool): Whether the response is acceptable.
        feedback (str): The feedback on the response.
    """
    is_acceptable: bool
    feedback: str


In [ ]:
EVALUATOR_SYSTEM_PROMPT = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {NAME} and is representing {NAME} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {NAME} in the form of their summary and LinkedIn details. Here's the information:"

EVALUATOR_SYSTEM_PROMPT += f"\n\n## Summary:\n{SUMMARY}\n\n## LinkedIn Profile:\n{LINKEDIN}\n\n"
EVALUATOR_SYSTEM_PROMPT += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply: str, message: str, history: list) -> str:
    """
    A function to create a user prompt for the evaluator.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
    Returns:
        str: The user prompt for the evaluator.
    """
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
def evaluate(reply: str, message: str, history: list) -> Evaluation:
    """
    Evaluate the response to a question.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
    Returns:
        Evaluation: The evaluation of the response.
    """

    messages = [{"role": "system", "content": EVALUATOR_SYSTEM_PROMPT}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
def rerun(reply: str, message: str, history: list, feedback: str) -> str:
    """
    Rerun the conversation with the updated system prompt.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
        feedback (str): The feedback on the response.
    Returns:
        str: The response from the Agent.
    """

    updated_system_prompt = SYSTEM_PROMPT + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message: str, history: list) -> str:
    """
    Chat with the agent.
    Args:
        message (str): The message to chat with the agent.
        history (list): The history of the conversation.
    Returns:
        str: The response from the agent.
    """
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    while not evaluation.is_acceptable:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
        evaluation = evaluate(reply, message, history)
    return reply
    

In [ ]:
# Create a chat interface
gr.ChatInterface(chat, type="messages").launch()